In [5]:
#下载minSearch
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

#下载数据
# !wget https://raw.githubusercontent.com/slavaheroes/llm-zoomcamp/refs/heads/homeworks/01-intro/documents.json

#下载得到数据的解析代码
# !wget https://raw.githubusercontent.com/slavaheroes/llm-zoomcamp/refs/heads/homeworks/01-intro/parse-faq.ipynb

--2025-08-19 09:19:14--  https://raw.githubusercontent.com/slavaheroes/llm-zoomcamp/refs/heads/homeworks/01-intro/parse-faq.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6986 (6.8K) [text/plain]
Saving to: ‘parse-faq.ipynb’

parse-faq.ipynb     100%[===================>]   6.82K  --.-KB/s    in 0s      

2025-08-19 09:19:15 (32.3 MB/s) - ‘parse-faq.ipynb’ saved [6986/6986]



In [4]:
import minsearch  # https://github.com/alexeygrigorev/minsearch
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)



In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [7]:
query = 'the course has already started, can I still enrol?'
results = index.search(
    query, 
    filter_dict={'course': 'data-engineering-zoomcamp'}, 
    boost_dict={'question': 3.0, 'section': 0.5}, 
    num_results=5
)

results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [9]:
#query = 'how do I run kafka?'
#search_results = index.search(query, filter_dict={'course': 'data-engineering-zoomcamp'}, boost_dict={'question': 3.0, 'section': 0.5}, num_results=5)

prompt = build_prompt(query, results)
prompt

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: the course has already started, can I still enrol?\n\n    CONTEXT: \n    section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\n

In [12]:
#通义千文大模型
from openai import OpenAI

import os
from openai import OpenAI
 
client = OpenAI(
    #api_key=os.getenv("DASHSCOPE_API_KEY"),
    api_key="sk-1c7c8651e8464b64b1b84ca3f182682d",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
 
completion = client.chat.completions.create(
    model="qwen-plus",  # 模型名称
    messages=[
        {"role": "system", "content": "你是教职助理，负责回答学生问题，用中文回答"},
        {"role": "user", "content": prompt}
    ],
    stream=False  # 是否流式输出
)
 
print(completion.choices[0].message.content)

Yes, you can still enrol and join the course after it has started. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's best not to leave everything for the last minute.


In [10]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [12]:
#elasticsearch 版本和 python elasticsearch 版本不对的问题
import sys
print(sys.path) 
# !{sys.executable} -m pip install elasticsearch==8.12.0
import elasticsearch
print(elasticsearch.__version__)
print(elasticsearch.__file__)

['/usr/local/python/3.12.1/lib/python312.zip', '/usr/local/python/3.12.1/lib/python3.12', '/usr/local/python/3.12.1/lib/python3.12/lib-dynload', '', '/home/codespace/.local/lib/python3.12/site-packages', '/usr/local/python/3.12.1/lib/python3.12/site-packages']
(8, 12, 0)
/usr/local/python/3.12.1/lib/python3.12/site-packages/elasticsearch/__init__.py


In [15]:
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/exNik8vTQ2KJZ0O-2Nybhw] already exists')

In [18]:
from tqdm import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████| 1006/1006 [00:02<00:00, 339.65it/s]


In [22]:
# def filter_search(query, size=5, search_words=[]):
#     def filter_builder():
#         return list(map(lambda word: {"term": {**word}}, search_words))
        
search_query = {
    "size": 2, # number of docs to be returned
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "hello world",
                    # Giving higher score for the document if the question of the document is much similar 
                    # to the query than the text of the document.
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            # "filter": filter_builder() # filter by the keyword (primary key)
        }
    }
}
es_client.search(index=index_name, body=search_query)
    # return es_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 482, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10, 'relation': 'eq'}, 'max_score': 10.8674, 'hits': [{'_index': 'course-questions', '_id': 'GfW_ypgBmRvvIiVr1ml2', '_score': 10.8674, '_source': {'text': 'Working on getting Docker installed - when I try running hello-world I am getting the error.\nDocker: Cannot connect to the docker daemon at unix:///var/run/docker.sock. Is the Docker daemon running ?\nSolution description\nIf you’re getting this error on WSL, re-install your docker: remove the docker installation from WSL and install Docker Desktop on your host machine (Windows).\nOn Linux, start the docker daemon with either of these commands:\nsudo dockerd\nsudo service docker start\nAdded by Ugochukwu Onyebuchi', 'section': '5. Deploying Machine Learning Models', 'question': 'Cannot connect to the docker daemon. Is the Docker daemon running?', 'course': 'machine-learning-zoomcamp'}}